# What hotel features are highly correlated?

## Extract data

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import sqlite3

In [2]:
# Connect to the database
conn = sqlite3.connect("Data/Hotels.db")

In [3]:
# Load the metadata2 table
meta = pd.read_sql("select * from metadata2;", conn)

# Load the ratings table
rate = pd.read_sql_query("select * from ratings;", conn)

In [4]:
# Close the connection
conn.close()

In [5]:
# Preview metadata table
meta.head()

,Name,Street,City,province,latitude,longitude,State,Stories,stars,airportDistance_km,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,Rancho Valencia Resort Spa,5921 Valencia Cir,Rancho Santa Fe,CA,32.990959000000004,-117.186136,California,nan,4.0,14.308848055373579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aloft Arundel Mills,7520 Teague Rd,Hanover,MD,39.155929,-76.716341,Maryland,7,4.0,4.668331572785505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Hampton Inn Suites PortlandVancouver,315 SE Olympia Dr,Vancouver,WA,45.619212,-122.525196,Washington,4,nan,6.591900084053486,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Hotel Phillips,106 W 12th St,Kansas City,MO,39.100119,-94.584701,Missouri,20,4.0,2.6706451419692976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,The Inn at Solvang,10611 Standing Stone Rd,Huntingdon,PA,40.527478,-77.969763,Pennsylvania,nan,nan,3.781816947263244,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
print(meta.shape)

(1853, 119)


In [7]:
# Drop columns from the rate table
rate = rate.drop(columns = ["index", "reviews_userCity", "reviews_userProvince", 
                            "reviews_text", "reviews_title", "reviews_sourceURLs"])

# Change the "name" to "Name"
rate = rate.rename(columns = {"name": "Name"})

# Convert review date from string to datetime format
rate["reviews_date"] = pd.to_datetime(rate["reviews_date"])

# Add a month-year column
rate["year"] = rate["reviews_date"].apply(lambda x: x.strftime("%Y"))

# Preview ratings table
rate.head()

,Name,reviews_date,reviews_rating,year
0,Rancho Valencia Resort Spa,2013-11-14,5.0,2013
1,Rancho Valencia Resort Spa,2014-07-06,5.0,2014
2,Rancho Valencia Resort Spa,2015-01-02,5.0,2015
3,Aloft Arundel Mills,2016-05-15,2.0,2016
4,Aloft Arundel Mills,2016-07-09,5.0,2016


## Transform data
In preparation for correlation analyses

### Merge data frames and remove sparse columns (with >70% null values)

In [8]:
# Merge the two dataframes
df = pd.merge(rate, meta, on = ["Name"])

# Convert "nan" to np.nan
df.replace("nan", np.nan, inplace = True)

# Remove columns that have been converted to dummy variables, not needed in analyses
df = df.drop(columns = ["reviews_date", "province", "Street", "City", "State", "latitude", "longitude"])

# Remove the columns with more than 70% null values 
df = df.dropna(thresh = 11572, axis = 1)

# Preview the data frame
df.head()

,Name,reviews_rating,year,airportDistance_km,airport,apartment,attractions,bars,beach,boutique,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
0,Rancho Valencia Resort Spa,5.0,2013,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Rancho Valencia Resort Spa,5.0,2014,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Rancho Valencia Resort Spa,5.0,2015,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aloft Arundel Mills,2.0,2016,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aloft Arundel Mills,5.0,2016,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Convert the year to dummy variables
encoded_year = pd.get_dummies(df["year"])

# Put the encoded year dataframe together with df
df1 = pd.concat([df, encoded_year], axis = 1)
df1 = df1.drop(columns = ["year"])
df1.head()

,Name,reviews_rating,airportDistance_km,airport,apartment,attractions,bars,beach,boutique,breakfast,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,Aloft Arundel Mills,2.0,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,Aloft Arundel Mills,5.0,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


### Remove columns with near zero variance

In [10]:
# Dependencies
from sklearn.feature_selection import VarianceThreshold

In [11]:
# Create a function that selects variables that pass the variance threshold
def variance_threshold_selector(data, threshold = 0.5):
    selector = VarianceThreshold()
    selector.fit_transform(data)
    
    return data[data.columns[selector.get_support(indices = True)]]

In [12]:
df2 = pd.concat([df1.iloc[:,0], variance_threshold_selector(df1.iloc[:,1:], 0.9)], axis = 1)
df2.head()

,Name,reviews_rating,airportDistance_km,airport,apartment,attractions,bars,beach,boutique,breakfast,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,Aloft Arundel Mills,2.0,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,Aloft Arundel Mills,5.0,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


### Are there categorical variables that are strongly correlated with each other?

In [13]:
# Create a dataframe just for correlation analyses
df3 = df2.drop(columns = ["Name", "reviews_rating", "airportDistance_km"], axis = 1)
df3.head()

,airport,apartment,attractions,bars,beach,boutique,breakfast,cabins,cable,campground,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [14]:
# from sklearn import matthews_corrcoef (calculates phi coefficient)
from sklearn.metrics import matthews_corrcoef

from IPython.core.display import clear_output

var_pairs = [(x, y) for x in df3.columns.values for y in df3.columns.values]

phi_list = []
for f in var_pairs:
    phi = matthews_corrcoef(list(df3[f[0]]), list(df3[f[1]]))
    phi_list.append(phi)    
        
    print(f"Calculating phi coefficients for {f}.")
    print("---")
    clear_output(wait = True)

Calculating phi coefficients for ('2018', '2018').
---


In [15]:
# Create a dataframe that contains tuples of variable pairs and phi coefficients
df5 = pd.DataFrame({"pair": var_pairs, "phi": phi_list})

# Split the tuples and put each variable in separate columns
df5[["var1", "var2"]] = pd.DataFrame(df5["pair"].tolist())

# Preview the dataframe
df5.head()

,pair,phi,var1,var2
0,"(airport, airport)",1.000000,airport,airport
1,"(airport, apartment)",-0.000728,airport,apartment
2,"(airport, attractions)",-0.000383,airport,attractions
3,"(airport, bars)",0.176531,airport,bars
4,"(airport, beach)",-0.001608,airport,beach


In [16]:
# Optional: create a correlation matrix
df6 = df5[["var1", "var2", "phi"]]
df6.pivot(index = "var1", columns = "var2", values = "phi")

var2,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,reservations,resort,restaurant,services,skiing,spas,theater,utility,village,wedding
var1,,,,,,,,,,,,,,,,,,,,,
2002,1.000000,-0.000086,-0.000148,-0.000284,-0.000388,-0.000684,-0.000591,-0.000757,-0.000579,-0.000916,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2003,-0.000086,1.000000,-0.000210,-0.000402,-0.000548,-0.000968,-0.000836,-0.001070,-0.000818,-0.001295,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2004,-0.000148,-0.000210,1.000000,-0.000696,-0.000950,-0.001677,-0.001449,-0.001854,-0.001418,-0.002243,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2005,-0.000284,-0.000402,-0.000696,1.000000,-0.001820,-0.003212,-0.002775,-0.003552,-0.002716,-0.004298,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2006,-0.000388,-0.000548,-0.000950,-0.001820,1.000000,-0.004387,-0.003791,-0.004851,-0.003710,-0.005871,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007,-0.000684,-0.000968,-0.001677,-0.003212,-0.004387,1.000000,-0.006689,-0.008560,-0.006546,-0.010359,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008,-0.000591,-0.000836,-0.001449,-0.002775,-0.003791,-0.006689,1.000000,-0.007396,-0.005656,-0.008951,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2009,-0.000757,-0.001070,-0.001854,-0.003552,-0.004851,-0.008560,-0.007396,1.000000,-0.007238,-0.011454,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2010,-0.000579,-0.000818,-0.001418,-0.002716,-0.003710,-0.006546,-0.005656,-0.007238,1.000000,-0.008759,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
# What variables to remove from the analyses? 
# Basis: correlation coefficient, r, is higher than 0.69 or lower than -0.69

high = []
for x in range(0, len(df5)):
    if df5["phi"][x] != 1 and df5["phi"][x] >= 0.7 or df5["phi"][x] <= -0.7:
        high.append(df5["pair"][x])
print(high)

[('hall', 'nightclub'), ('motel', 'reservations'), ('nightclub', 'hall'), ('reservations', 'motel')]


In [18]:
# Remove a variable that is highly correlated with the other from dataframe df2
df4 = df2.drop(columns = ["hall", "motel"], axis = 1)
df4.head()

,Name,reviews_rating,airportDistance_km,airport,apartment,attractions,bars,beach,boutique,breakfast,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,Aloft Arundel Mills,2.0,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
4,Aloft Arundel Mills,5.0,4.668331572785505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


## Load the df2 dataframe with new information into a new database table 

In [19]:
# Create a connection to the database
conn = sqlite3.connect("Data/Hotels.db")

# Save the dataframe as a sqlite database table
df4.to_sql("alldata", conn, if_exists = "replace", index = False)

In [20]:
# Preview metadata
pd.read_sql_query("select * from alldata limit 3;", conn)

,Name,reviews_rating,airportDistance_km,airport,apartment,attractions,bars,beach,boutique,breakfast,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
2,Rancho Valencia Resort Spa,5.0,14.308848055373579,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [21]:
conn.close()